In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statistics import mode

%matplotlib inline

In [ ]:
df = pd.read_csv(r"C:\Users\ramya\Downloads\archive (13)\amz_uk_price_prediction_dataset.csv")

df

# Part 1: 

## Analyzing Best-Seller Trends Across Product Categories
**Objective: Understand the relationship between product categories and their best-seller status.**

**Crosstab Analysis:**

- Create a crosstab between the product category and the isBestSeller status.
- Are there categories where being a best-seller is more prevalent?
- Hint: one option is to calculate the proportion of best-sellers for each category and then sort the categories based on this proportion in descending order.

**Statistical Tests:**

- Conduct a Chi-square test to determine if the best-seller distribution is independent of the product category.
- Compute Cramér's V to understand the strength of association between best-seller status and category.

**Visualizations:**

- Visualize the relationship between product categories and the best-seller status using a stacked bar chart.

In [ ]:
crosstab_result = pd.crosstab(df['category'], df['isBestSeller']).sort_values(True, ascending = False)
crosstab_result

In [ ]:
category_proportions = df.groupby('category')['isBestSeller'].mean().sort_values(ascending = False).head(20)
category_proportions

In [ ]:
from scipy.stats import chi2_contingency

# Chi-square test for 'MSZoning' and 'SaleCondition'
chi2_statistic, chi2_p_value, _, _ = chi2_contingency(crosstab_result)

chi2_statistic, chi2_p_value

In [ ]:
from scipy.stats.contingency import association

association(crosstab_result, method="cramer")

In [ ]:
crosstab_result.head(10).plot(kind="bar", stacked=True)
plt.xticks(rotation = 90)

# Part 2: 

## Exploring Product Prices and Ratings Across Categories and Brands
**Objective: Investigate how different product categories influence product prices.**

**Preliminary Step: Remove outliers in product prices.**

- For this purpose, we can use the IQR (Interquartile Range) method. Products priced below the first quartile minus 1.5 times the IQR or above the third quartile plus 1.5 times the IQR will be considered outliers and removed from the dataset. The next steps will be done with the dataframe without outliers.
- Hint: you can check the last Check For Understanding at the end of the lesson EDA Bivariate Analysis for a hint on how to do this.

**Bar Charts:**

- Create a bar chart comparing the average price of products for the top 10 product categories (based on count).
- Which product category commands the highest average price? Don't filter here by top categories.

**Violin Plots:**

- Use a violin plot to visualize the distribution of price across different product categories. Filter out the top 20 categories based on count for better visualization.
- Which product category commands the highest average price? Don't filter here by top categories.

**Box Plots:**

- Visualize the distribution of product ratings based on their category using side-by-side box plots. Filter out the top 10 categories based on count for better visualization.
- Which category tends to receive the highest median rating from customers? Don't filter here by top categories.

In [ ]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
    
cleaned_data = df[(df['price'] > lower_bound) & (df['price'] < upper_bound)]


In [ ]:
cleaned_data

In [ ]:
top_20_category = cleaned_data['category'].value_counts().sort_values(ascending = False).head(20).index
top_20_category

In [ ]:
df_top_20 = cleaned_data[cleaned_data['category'].isin(top_20_category)]
df_top_20

In [ ]:
sns.violinplot(x='category', y='price', data=df_top_20)
plt.xticks(rotation=90)

In [ ]:
cleaned_data.groupby('category')['price'].agg('mean').sort_values(ascending = False).head(20)

- Visualize the distribution of product ratings based on their category using side-by-side box plots. Filter out the top 10 categories based on count for better visualization.
- Which category tends to receive the highest median rating from customers? Don't filter here by top categories.

In [ ]:
sns.barplot(data = df_top_10, x = 'category', y = 'price')
plt.xticks(rotation = 90)

In [ ]:
top_10_category = cleaned_data['category'].value_counts().sort_values(ascending = False).head(10).index
df_top_10 = cleaned_data[cleaned_data['category'].isin(top_10_category)]
df_top_10

In [ ]:
sns.boxplot(data=df_top_10, x='category', y='stars', palette="coolwarm")
plt.xticks(rotation=90)

# Part 3: 
## Investigating the Interplay Between Product Prices and Ratings

**Objective:** 
    Analyze how product ratings (stars) correlate with product prices.

**Correlation Coefficients:**

- Calculate the correlation coefficient between price and stars.
- Is there a significant correlation between product price and its rating?

**Visualizations:**

- Use a scatter plot to visualize the relationship between product rating and price. What patterns can you observe?
- Use a correlation heatmap to visualize correlations between all numerical variables.
- Examine if product prices typically follow a normal distribution using a QQ plot.


In [ ]:
correlation = cleaned_data['stars'].corr(cleaned_data['price'])
correlation

In [ ]:
correlation = cleaned_data['stars'].corr(cleaned_data['price'], method='spearman')
correlation

- Pearson Correlation: (-0.08)
- Spearman Correlation: (-0.07)

In [ ]:
sns.scatterplot(data=cleaned_data, x='stars', y='price')

In [ ]:
correlation_matrix = cleaned_data.corr()

plt.figure(figsize=(18, 15))

sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")

plt.title("Correlation Heatmap for Selected Numerical Variables")
plt.show()

In [ ]:
import statsmodels.api as sm

sm.qqplot(cleaned_data['price'], line='s');